결측치 처리 방법론

결측값은 데이터 셋이서 비어 있거나(Null), NA(Not Available)  
또는 NaN(Not a Number)으로 저장된 값들입니다.

결측값이 있는 데이터를 사용하여 데이터 분석을 수행할 경우   
일반적으로 에러 메시지가 발생하며 많은 ML 알고리즘은 이를 허용하지 않습니다. 
따라서 이러한 결측값은 고정된 다른 값으로 변환해야 합니다. 

결측값을 어떻게 처리해야 할지는 경우에 따라 다릅니다.
피처 값 중 결측값이 얼마 되지 않는다면 피처의 평균값 등으로  
간단히 대체할 수 있습니다.
하지만 결측값이 대부분이라면 오히려 해당 피처는 드롭하는 것이  
더 좋은 경우도 있습니다.
이는 정확히 몇 퍼센트의 결측값을 포함하하는 것이 좋은가에 대해   
정의내릴 수 없으며 상황에 따른 분석가의 주관적인 판단이 들어갑니다.  
  
  
1) 행 삭제(Row Delete)  
결측값이 있는 관측값(행)을 제거  
각 행이 독립이면 삭제해도 무방하나  
독립이 아니면(시간 순서로 된 데이터) 다른 값으로 대체하거나  
행의 단위를 바꾸는등 다른 방법을 고려해야함

결측치를 처리함에 있어서 경우에 따라 처리 방법이 달라집니다. 

결측치를 처리하는 여러가지 방법 중에 가장 단순한 방법은  
행을 삭제하는 방법입니다. 

이 방법은 샘플 수가 충분히 많다면 missing value 를 포함하는 행을  
모두 삭제하는 것이 가능합니다. 


In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
 
df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                "toy": [np.nan, 'Batmobile', 'Bullwhip'],
                 "born": [pd.NaT, pd.Timestamp("1940-04-25"),pd.NaT]})
df

,name,toy,born
0,Alfred,NaN,NaT
1,Batman,Batmobile,1940-04-25
2,Catwoman,Bullwhip,NaT


In [2]:
# 결측치가 하나라도 있으면 버리는 코드 예제
df.dropna()

,name,toy,born
1,Batman,Batmobile,1940-04-25


In [3]:
# 모든 값이 Null일 경우만 버리는 코드 예제
df.dropna(how='all')

,name,toy,born
0,Alfred,NaN,NaT
1,Batman,Batmobile,1940-04-25
2,Catwoman,Bullwhip,NaT


In [4]:
# 결측치가 하나 이상 있는 Case만 선택하는 코드 예제
df[df.isnull().any(axis=1)]

,name,toy,born
0,Alfred,NaN,NaT
2,Catwoman,Bullwhip,NaT


2) 단순 대체(Single Imputation)

최빈값 : 범주형 데이터, 가장 많이 등장하는 값으로 대체  
mean/median : 수치형 데이터, 평균/중위수로 대체  
hot-deck : 결측값이 아닌 값들 중에서 랜덤하게 추출하여 대체  
regression : 결측값이 아닌 값들 및 기타 변수로 회귀모델을 생성하여 결측값을 추정하여 대체   

대체기법은 해당 변수의 분포 및 다른 변수와의 관계를 변경시킬 수 있으므로 선택에 신중해야 함  



3) 다중 대체(Multiple Imputation)

결측값이 아닌 나머지 변수를 사용하여 결측값을 대체하는 작업을 여러번 반복하여 나오는 결과를 통합함  

만약 샘플수가 충분하지 않을 경우, Pandas의 fillna() 명령어로 Null 값을 채우는 것이 가능합니다. 

* 연속형인 경우 Mean이나 Median 등 대푯값 이용  
* 명목형인 경우 Mode(최빈치)나 예측 모형을 통해 Null 값 대체  
경우에 따라 도메인 지식을 이용해 결측치를 처리하거나, 회귀 및 분류 예측모델을 이용하는 경우도 있습니다.

In [8]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns=list('ABCD'))
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


In [7]:
# Null 값을 mean으로 대체하는 코드 예제
df.fillna(df.mean())

,A,B,C,D
0,3.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,3.0,NaN,5
3,3.0,3.0,NaN,4



# 결측치를 처리할 때 고려할 점

결측치를 처리할 경우에도 도메인 지식은 유용하게 사용됩니다.

인적, 기계적 원인임이 판명되면, 협업자와 지속적으로 노력해   
결측치를 사전에 발생하지 않도록 조치하는 것이 좋습니다.

수치형인 경우 의미상으로 0으로 메꾸는 것이 맞는지 아니면 평균이나  
중앙치가 맞는지 등은 데이터에 대한 배경지식이 있는 경우,   
보다 적절한 의사결정을 할 수 있습니다.

예를 들어 viewCount가 1이상인데, edit, export가 missing인 경우   
(도메인 지식을 통해) 0으로 메꾸는 것이 가능합니다.

View 가 다른치 행동에 선행하는 개념이기 때문에   
위와 같은 의사결정이 가능합니다.

특히 숫자 0과 null 과 같은 결측치는 완전히 다른 개념이니   
유의해야 합니다.

만약 target(group)에 결측치가 있다면 imputation이 아닌 drop 으로  
처리하도록 합니다.

* 0: -1과 1 사이의 가운데 숫자(정수)
* null: 미지의 값

In [14]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns=list('ABCD'))
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


In [19]:
# Null이 아닌 값을 앞뒤로 전파
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,4.0,NaN,5
3,3.0,3.0,NaN,4


In [18]:
# 각 컬럼마다 모든 NaN값 지정 변경
values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
df.fillna(value=values)

,A,B,C,D
0,0.0,2.0,2.0,0
1,3.0,4.0,2.0,1
2,0.0,1.0,2.0,5
3,0.0,3.0,2.0,4


In [20]:
# 각 컬럼마다 첫번째 NaN값 지정 변경
df.fillna(value=values, limit=1)

,A,B,C,D
0,0.0,2.0,2.0,0
1,3.0,4.0,NaN,1
2,NaN,1.0,NaN,5
3,NaN,3.0,NaN,4


# 이상치 처리 방법론
이상치는 다른 관측값들과 다른 형태를 갖거나 패턴에서 벗어난 데이터를 말합니다.  
이상치가 발생하게되는 원인에는 여러가지가 있지만 분석가 입장에서  
이를 어떻게 바라보고 처리해야 하는지에 대한 판단은 매우 중요합니다.  
이상치 발생 원인을 크게 두가지 관점에서 바라 볼 수 있습니다.  
첫번째는 데이터 수집 시 관측 오류 또는 설비 등의 오작동에 의한  
생성된 경우입니다. 이러한 경우에는 현업과 협력하여 명확한 원인을 찾고  
재발 가능성을 진단한 후 분석에는 제거하는 것이 바람직 합니다.  

두번째는 개체의 특성에 의한 이상치 입니다. 이는 분석 결과를 변화시킬  
수준이 아니라면 개체들의 분포 및 특성을 반영하기 위해 가급적 제거하지  
않는 것이 좋습니다.  

이상치 처리 일반적으로 아래와 같은 3가지 경우를 말합니다.  
1) 표준점수로 변환 후 -3 이하 및 +3 제거  
2) IQR 방식  
3) 도메인 지식 이용하거나 Binning 처리하는 방식  

표준점수 이용할 경우 평균이 0, 표준편차가 1인 분포로 변환한후  
+3 이상이거나 -3 이하인 경우 극단치로 처리합니다


In [21]:
# 표준점수 기반 예제 코드
def std_based_outlier(df):
    for i in range(0, len(df.iloc[1])):
        df.iloc[:,i] = df.iloc[:,i].replace(0, np.NaN) # optional
        df = df[~(np.abs(df.iloc[:,i] - df.iloc[:,i].mean()) > (3*df.iloc[:,i].std()))].fillna(0)

IQR 방식은 75% percentile + 1.5 * IQR 이상이거나  
25 percentile - 1.5 * IQR 이하인 경우  
극단치로 처리하는 방식입니다.

이해하기 쉽고 적용하기 쉬운 편이지만,  
경우에 따라 너무 많은 사례들이 극단치로 고려되는 경우가 있으니  
주의할 필요가 있습니다.

IQR 및 outliers를 보기 쉽게 시각화한 box plot 을 활용하여  
변수의 분포를 확인 후 이상치를 처리할 수 도 있습니다. 

이때 분석가의 판단에 앞서서 현장 전문가와의 상의를 통하여  
이상치를 처리하는 것이 바람직합니다.

* 개별 컬럼들의 분포로 도출되지 않은 이상치는 산점도를 이용하여 찾아냄.  
* 단, 다른 인자로 인한 특이값이 이상치처럼 보일 수 있으므로   
  발생가능한 값인지 현장 전문가(현업)와 논의 후 제거여부를 결정함

# 변수 변환
앞서서 설명한 결측치를 처리하거나 이상치를 처리하는 방법과 다르게   
변수 변환은 주어진 원 자료(데이터)를 그대로 사용하지 않고   
다른 형태로 변환시키는 방법입니다. 

이러한 변수 변환과 같은 전처리 과정을 통해 데이터의 탐색과   
분석을 용이하게 만들어 줍니다.

## 데이터 분포 변환 

대부분의 분석 모델은 변수가 특정 분포를 따른다는 가정을 기반으로 합니다.  
예를 들어 선형 모델의 경우, 설명 및 종속변수 모두가 정규분포와   
유사할 경우 성능이 높아지는 것으로 알려져 있습니다.  

자주 쓰이는 방법은 Log, Exp, Sqrt 등 함수를 이용해 데이터 분포를  
변환하는 것입니다.

In [22]:
import math
from sklearn import preprocessing
 
df = pd.DataFrame(np.array([1, 2, 3, 4, 7, 11, 12, 103, 104, 1005, 1006, 1007]),columns=['X'])
df

,X
0,1
1,2
2,3
3,4
4,7
5,11
6,12
7,103
8,104
9,1005


In [29]:
# 특정 변수에만 함수 적용
df['X_log'] = preprocessing.scale(np.log(df['X']+1)) # 로그
df['X_sqrt'] = preprocessing.scale(np.sqrt(df['X']+1)) # 제곱근
df

,X,X_log,X_sqrt
0,1,-1.221285,-0.787971
1,2,-1.044650,-0.762052
2,3,-0.919326,-0.740202
3,4,-0.822117,-0.720952
4,7,-0.617367,-0.672647
5,11,-0.440733,-0.620810
6,12,-0.405864,-0.609276
7,103,0.500012,-0.071684
8,104,0.504181,-0.067695
9,1005,1.488617,1.683145


In [30]:
# 데이터 프레임 전체에 함수 적용 (단, 숫자형 변수만 있어야 함)
df_log = df.apply(lambda x: np.log(x+1)) 
df_log

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,X,X_log,X_sqrt
0,0.693147,NaN,-1.551030
1,1.098612,NaN,-1.435704
2,1.386294,-2.517340,-1.347851
3,1.609438,-1.726630,-1.276370
4,2.079442,-0.960680,-1.116716
5,2.484907,-0.581128,-0.969718
6,2.564949,-0.520647,-0.939753
7,4.644391,0.405473,-0.074383
8,4.653960,0.408249,-0.070096
9,6.913737,0.911727,0.986989


위 방법 외에도 분포의 특성에 따라 제곱, 자연로그, 지수 등 다양한 함수가 사용될 수 있습니다.
* left_distribution: X^3
* mild_left: X^2
* mild_right: sqrt(X)
* right: ln(X)
*  servere right: 1/X

## 1) Log·Sqrt Transformation
원 자료(raw data)에 log·sqrt를 취하여  
-데이터를 선형(linear)으로 만들고  
-좌우 대칭인 분포로(정규분포에 가깝게) 만들어줌  
-분산을 일정하게 만들어주기도 함  

*log 또는 sqrt를 취할 때는 음수나 0에 주의  
(상수를 더한 후 마지막에 다시 빼는 방법을 사용)

### 데이터 단위 변환  

데이터의 스케일(측정단위)이 다를 경우 특히 거리를 기반으로  
분류하는 모델(KNN 등)에 부정적인 영향을 미치게 됩니다.  
따라서 스케일링을 통해 단위를 일정하게 맞추는 작업을 진행해야 합니다.  
아래 방식이 주로 스케일링을 위해 쓰이는 방법입니다.  
대부분의 통계 분석 방법이 정규성 가정을 기반으로 하므로  
완벽하지 않더라도 최대한 정규분포로 변환하는 노력이 필요합니다.  
* Scaling: 평균이 0, 분산이 1인 분포로 변환  
* MinMax Scaling: 특정 범위 (예, 0~1)로 모든 데이터를 변환  
* Box-Cox: 여러 k 값중 가장 작은 SSE 선택  
* Robust_scale: median, interquartile range 사용(outlier 영향 최소화)  

In [32]:
from scipy.stats import boxcox
 
df = pd.DataFrame(np.array([[ 1., -1.,  2.],
                       [ 2.,  0.,  0.],
                      [ 0.,  1., -1.]]),columns=['X','Y','Z'])
df

,X,Y,Z
0,1.0,-1.0,2.0
1,2.0,0.0,0.0
2,0.0,1.0,-1.0


In [33]:
# 변수별 scaling 적용
df['X_scale'] = preprocessing.scale(df['X'])
df['X_boxcox'] = preprocessing.scale(boxcox(df['X']+1)[0])
df['X_robust_scale'] = preprocessing.robust_scale(df['X'])
df

,X,Y,Z,X_scale,X_boxcox,X_robust_scale
0,1.0,-1.0,2.0,0.000000,0.090648,0.0
1,2.0,0.0,0.0,1.224745,1.176903,1.0
2,0.0,1.0,-1.0,-1.224745,-1.267550,-1.0


* 표준화 (Standardization)  
관측값이 평균으로부터 얼마나 떨어져있는지를 나타낼 때 사용  
평균을 빼고 표준편차로 나누어줌  
평균을 0, 분산을 1로 변환하여(Z-score)  변수들 간 단위가 다를 때  
기준을 맞춰주는 역할을 함
* 정규화 (Normalization)  
관측값이 전체 값의 범위(range) 중 어느 정도 위치를 차지하는지  
나타낼 때 사용  
최소값을 빼고 [최대값-최소값]으로 나누어줌  
0과 1 사이의 값으로 표현됨
* 주기를 갖는 데이터의 변동을 확인할 때 유용함  
(과거 대비 현재 값이 얼마나 변화했는지 일관된 기준으로 확인 가능)

# Dummy Variable
범주형 데이터는 회귀분석이나 여러 ML 알고리즘에 바로 적용하여  
사용할 수 없습니다. 이를 해결하기 위한 방법으로 범주형 변수를  
0 또는 1의 수치형 변수로 변환해주어야 합니다.
이는 범주형 변수에 대한 모든  
정보를 활용하여 모델링에 사용하기 위한 목적에 있습니다.  
one-hot encoding과 비슷한 개념이나, dummy variable의 경우에는  
[범주의 개수-1]개의 변수로 변환하고 one-hot encoding의 경우에는  
[범주의 개수]개의 변수로 변환한다는 차이점이 있습니다.

In [35]:
import numpy as np
import pandas as pd
 
season = pd.DataFrame({'season':['spring', 'summer', 'fall', 'winter', np.nan]})
season['season']

0    spring
1    summer
2      fall
3    winter
4       NaN
Name: season, dtype: object

In [36]:
# pd.get_dummies 처리 : 결측값을 제외하고 0과 1로 구성된 더미값 생성
pd.get_dummies(season['season'])

,fall,spring,summer,winter
0,0,1,0,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,0


In [37]:
# 결측값 처리(dummy_na = True 옵션) :  Nan을 생성하여 결측값도 인코딩하여 처리
pd.get_dummies(season['season'], dummy_na=True)

,fall,spring,summer,winter,NaN
0,0,1,0,0,0
1,0,0,1,0,0
2,1,0,0,0,0
3,0,0,0,1,0
4,0,0,0,0,1
